In [1]:
import torch
from torch import nn
import nltk
import matplotlib as plt
import numpy as np 
import argparse
import pickle 
import os
from torch.autograd import Variable 
from torchvision import transforms 
from build_vocab import Vocabulary
from model_new import EncoderCNN, DecoderRNN
from PIL import Image
from data_loader import CocoDataset
from collections import Counter
from pycocotools.coco import COCO
from step_1 import encode,decode

In [2]:
def to_var(x, volatile=False):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x, volatile=volatile)

def load_image(image_path, transform=None):
    image = Image.open(image_path)
    image = image.resize([224, 224], Image.LANCZOS)
    
    if transform is not None:
        image = transform(image).unsqueeze(0)
    
    return image

In [3]:
transform = transforms.Compose([
        transforms.ToTensor(), 
        transforms.Normalize((0.485, 0.456, 0.406), 
                             (0.229, 0.224, 0.225))])
with open('./data/vocab.pkl', 'rb') as f:
    vocab = pickle.load(f)

# Build Models
encoder = EncoderCNN(256)
encoder.eval()  # evaluation mode (BN uses moving mean/variance)
decoder = DecoderRNN(256, 512, 
                     len(vocab), 1)

# Load the trained model parameters
encoder.load_state_dict(torch.load('./models/encoder_pretrained.pkl'))
decoder.load_state_dict(torch.load('./models/decoder_pretrained.pkl'))

In [4]:
image_id = "1"
image = load_image('data/step_1/image_'+image_id+'.jpg', transform)
image_tensor = to_var(image, volatile=True)
# If use gpu
if torch.cuda.is_available():
    encoder.cuda()
    decoder.cuda()

/home/AD/b3pang/anaconda2/lib/python2.7/site-packages/torch/backends/cudnn/__init__.py:48: UserWarning: PyTorch was compiled without cuDNN support. To use cuDNN, rebuild PyTorch making sure the library is visible to the build system.
  "PyTorch was compiled without cuDNN support. To use cuDNN, rebuild "


In [5]:
feature = encoder(image_tensor)

In [6]:
pred,predu = decode(feature,vocab.word2idx['<start>'],decoder,vocab, c_step=2.0,prop_step=-1)
print pred
print predu

<start> a plane flying through the air with a sky background <end>
<start> a plane flying through the air with a sky background <end>


In [9]:
teach_wordid = ['<start>','several','planes']
print [vocab.word2idx[word] for word in teach_wordid]
pred,predu = decode(feature,[vocab.word2idx[word] for word in teach_wordid],decoder,vocab, c_step=2.0,prop_step=-1)
print pred
print predu

[1, 4525, 1673]
******
2
******
2
******
1
<pad> <pad> <pad> are parked in a field with a sky background <end>
<pad> <pad> <pad> are parked in a field with a sky background <end>


In [1]:
from model import DecoderRNN

In [1]:
teach_wordid = ['<start>','several','planes']
pred,predu = decode(feature,[vocab.word2idx[word] for word in teach_wordid],decoder,vocab, c_step=2.0,prop_step=-1)
print pred
print predu

NameError: name 'decode' is not defined

In [12]:
loss = nn.L1Loss()
input = Variable(torch.randn(3, 5), requires_grad=True)
print input.grad
target = Variable(input.data+2)
output = loss(input, target)
output.backward()
print input.grad
input.grad.data.zero_()
print input.grad

None
Variable containing:
1.00000e-02 *
 -6.6667 -6.6667 -6.6667 -6.6667 -6.6667
 -6.6667 -6.6667 -6.6667 -6.6667 -6.6667
 -6.6667 -6.6667 -6.6667 -6.6667 -6.6667
[torch.FloatTensor of size 3x5]

Variable containing:
 0  0  0  0  0
 0  0  0  0  0
 0  0  0  0  0
[torch.FloatTensor of size 3x5]



In [23]:
target

Variable containing:
 1.7598  2.8590  0.9656  1.5102  3.7906
 0.7517  1.2949  0.6209  1.3271  0.7654
 0.5926  2.5933  1.2873  3.1093  2.4338
[torch.FloatTensor of size 3x5]